# Report

## Introduction/Business Problem

### Problem
Develop a "Living Index" - a rating for each neighbourhood of Toronto that uses combination of factors that influence how suitable the area is for living there. For instance, "Living Index" for The Beaches could be 9.1 (out of 10), whereas "Rustic" could have 3.2.

The developed system would help people that plan to move to Toronto to select area where to live.

### Target audience
Any user around the world that intends to move to Toronto, or just curious.

### Living Index
Living index will combine a number of factors to come up with overall result. Those factors (including, but not limited to): distance to malls, distance to schools (rated), distance to food points (cafes, restaurants, etc.), public transport, crime rates, etc.

### Leverage Foursquare
Wherever possible, Foursquare will be used to retrieve data for neighbourhoods (such as school locations, restaurants, cinemas, etc.).

## Data

1. Neighbourhood data with locations (scraped from Wikipedia and with geo data added).

    Example:

    ![Neighbourhood data](images/neigh_data.png)

2. Foursquare

    Foursquare API will be used to retrieve data about locations of important businesses, etc.

    * School locations with reviews
    * Restaurant locations with reviews
    * Businesses locations

    Example:

    ```bash
    curl -X GET -G \
        'https://api.foursquare.com/v2/venues/explore' \
            -d client_id="CLIENT_ID" \
            -d client_secret="CLIENT_SECRET" \
            -d v="20180323" \
            -d ll="40.7243,-74.0018" \
            -d query="coffee" \
            -d limit=1
    ```

3. Toronto Police Data from Kaggle:
    [Reports and files on Major Crimes and Location](https://www.kaggle.com/kapastor/toronto-police-data-crime-rates-by-neighbourhood)

    This dataset contains records for crimes that ocurred in Toronto from 2014 to 2019, with the information on their locations and types.

    Example:

    ![crime data](images/crime_data.png)